In [72]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


In [73]:
df_csv = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

In [74]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


In [75]:
#split the data 
training_data, testing_data = train_test_split(df_csv,random_state = 2000)

In [76]:
# GET LABELS
Y_train=training_data['target'].values
Y_test=testing_data['target'].values

In [77]:
def extract_features(df,field,training_data,testing_data,type="binary"):
    """Extract features using different methods"""
    
    if "binary" in type:
        
        # BINARY FEATURE REPRESENTATION
        cv= CountVectorizer(binary=True, max_df=0.5)
        cv.fit_transform(training_data[field].values)
        
        train_feature_set=cv.transform(training_data[field].values)
        test_feature_set=cv.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,cv
  
    elif "counts" in type:
        
        # COUNT BASED FEATURE REPRESENTATION
        cv= CountVectorizer(binary=False, max_df=0.95)
        cv.fit_transform(training_data[field].values)
        
        train_feature_set=cv.transform(training_data[field].values)
        test_feature_set=cv.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,cv
    
    else:    
        
        # TF-IDF BASED FEATURE REPRESENTATION
        tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.95)
        tfidf_vectorizer.fit_transform(training_data[field].values)
        
        train_feature_set=tfidf_vectorizer.transform(training_data[field].values)
        test_feature_set=tfidf_vectorizer.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,tfidf_vectorizer

In [78]:
X_train,X_test,feature_transformer=extract_features(df_csv,"text",training_data,testing_data,type="binary")

In [79]:
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=2, penalty='l2',max_iter=10000)
model=scikit_log_reg.fit(X_train,Y_train)

[LibLinear]

In [80]:
model.score(X_test, Y_test)

0.7909663865546218

In [81]:
model.predict(feature_transformer.transform(["Arsonist Sets NYC Vegetarian Restaurant on Fire: Police #NewYork - http://t.co/Nr7usT3uh8"]))

array([1])

In [82]:
# 0 for not a disaster
# 1 for a disaster
model.classes_ 

array([0, 1])

In [ ]:
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
arr = df_test["text"].tolist()
arr[0]

In [ ]:
arr_op = model.predict(feature_transformer.transform(arr))

In [ ]:
df_test.drop(axis=1,labels=["keyword","location","text"], inplace =True)

In [ ]:
df_test["target"] = arr_op

In [ ]:
df_test

In [ ]:
df_test.to_csv("output.csv", index=False)